# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [2]:
#Your code here; import the necessary packages
import pandas as pd
import sqlite3

In [3]:
#Your code here; create the connection and cursor
conn = sqlite3.connect('data.sqlite')
c = conn.cursor()

## Select the Average Number of Orders on a Product by Product Basis

In [3]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""SELECT AVG(num_orders) FROM 
                                (SELECT COUNT(productCode) as num_orders from products
                                                        join orderdetails
                                                        using (productCode)
                                                        group by productCode);
                """)
d = pd.DataFrame(c.fetchall())
d.columns = [x[0] for x in c.description]
d

,AVG(num_orders)
0,27.486239


## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [11]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""SELECT customerNumber, contactLastName, contactFirstName
                                    FROM customers
                                    WHERE (SELECT customerNumber FROM orders
                                            WHERE orderDate = '2003-01-31')
                                    GROUP BY customerNumber; """)
d = pd.DataFrame(c.fetchall())
d.columns = [x[0] for x in c.description]
d

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [51]:
#Your code here; use a subquery. No join will be necessary.
c.execute("""SELECT productName, COUNT(orderNumber) as num_orders
                            FROM (SELECT productName, orderNumber FROM products
                                            LEFT JOIN orderdetails
                                            USING (productCode))
                            GROUP BY productName
                            ORDER BY num_orders DESC;""")

d = pd.DataFrame(c.fetchall())
d.columns = [x[0] for x in c.description]
d.head()

,productName,num_orders
0,1992 Ferrari 360 Spider red,53
1,18th Century Vintage Horse Carriage,28
2,1900s Vintage Bi-Plane,28
3,1900s Vintage Tri-Plane,28
4,1913 Ford Model T Speedster,28


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [53]:
#Your code here
c.execute("""SELECT productName, COUNT(distinct customerNumber) as num_people_ordered
                            FROM products
                            JOIN orderdetails
                            USING (productCode)
                            JOIN orders
                            USING (orderNumber)
                            GROUP BY productName
                            ORDER BY num_people_ordered DESC;""")

d = pd.DataFrame(c.fetchall())
d.columns = [x[0] for x in c.description]
d.head()

,productName,num_people_ordered
0,1992 Ferrari 360 Spider red,40
1,1934 Ford V8 Coupe,27
2,1952 Alpine Renault 1300,27
3,1972 Alfa Romeo GTA,27
4,Boeing X-32A JSF,27


## Select the Employee Number, Office Code, City (of the office), and Name (First and Last) of those Employees who Sold Products that Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [35]:
c.execute("""SELECT productName, COUNT(distinct customerNumber)
                                                FROM products
                                                JOIN orderdetails 
                                                USING (productCode)
                                                JOIN orders
                                                USING (orderNumber)
                                                JOIN customers
                                                USING (customerNumber)
                                                JOIN employees 
                                                ON customers.salesRepEmployeeNumber = employees.employeeNumber
                                                GROUP BY productCode
                                                ORDER BY COUNT(distinct customerNumber) DESC;""")

d = pd.DataFrame(c.fetchall())
d.columns = [x[0] for x in c.description]
d.head()

,productName,COUNT(distinct customerNumber)
0,1992 Ferrari 360 Spider red,40
1,1952 Alpine Renault 1300,27
2,1972 Alfa Romeo GTA,27
3,1934 Ford V8 Coupe,27
4,Boeing X-32A JSF,27


In [40]:
#mine
c.execute("""SELECT DISTINCT employeeNumber,
                    officeCode,
                    city,
                    firstName, 
                    lastName
                    FROM employees 
                    JOIN offices
                    USING (officeCode)
                    WHERE employeeNumber in (SELECT salesRepEmployeeNumber as employeeNumber
                                                FROM products
                                                JOIN orderdetails 
                                                USING (productCode)
                                                JOIN orders
                                                USING (orderNumber)
                                                JOIN customers
                                                USING (customerNumber)
                                                JOIN employees 
                                                ON customers.salesRepEmployeeNumber = employees.employeeNumber
                                                GROUP BY productCode
                                                HAVING COUNT(distinct customerNumber) <= 20);""")

d = pd.DataFrame(c.fetchall())
d.columns = [x[0] for x in c.description]
print(d.employeeNumber.unique().shape)

(4,)


In [41]:
#solution
c.execute("""select distinct employeeNumber,
                    officeCode,
                    o.city,
                    firstName,
                    lastName
                    from employees e
                    join offices o
                    using(officeCode)
                    join customers c
                    on e.employeeNumber = c.salesRepEmployeeNumber
                    join orders
                    using (customerNumber)
                    join orderdetails
                    using (orderNumber)
                    where productCode in (select productCode
                                                 from products
                                                 join orderdetails
                                                 using (productCode)
                                                 join orders
                                                 using (orderNumber)
                                                 group by 1
                                                 having count(distinct customerNumber) <= 20
                                         );
           """
         )
d2 = pd.DataFrame(c.fetchall())
d2.columns = [x[0] for x in c.description]
print(d2.employeeNumber.unique().shape)

(15,)


In [43]:
print(d.employeeNumber.unique(),d2.employeeNumber.unique())

['1216' '1370' '1611' '1612'] ['1370' '1401' '1621' '1501' '1337' '1286' '1166' '1165' '1612' '1188'
 '1611' '1323' '1702' '1504' '1216']


In [15]:
c.execute("""SELECT employeeNumber, officeCode, city, firstName, lastName
                            FROM employees 
                            JOIN offices
                            USING (officeCode);
                            
          
                            WHERE employeeNumber in (SELECT salesRepEmployeeNumber 
#                                                         FROM customers
#                                                         JOIN orders
#                                                         USING (customerNumber)
#                                                         JOIN orderdetails 
#                                                         USING (orderNumber)
#                                                         JOIN products 
#                                                         USING (productCode)
#                                                         GROUP BY salesRepEmployeeNumber
#                                                         HAVING COUNT(customerNumber) < 20);""")

In [16]:
c.fetchall()

[('1002', '1', 'San Francisco', 'Diane', 'Murphy'),
 ('1056', '1', 'San Francisco', 'Mary', 'Patterson'),
 ('1076', '1', 'San Francisco', 'Jeff', 'Firrelli'),
 ('1088', '6', 'Sydney', 'William', 'Patterson'),
 ('1102', '4', 'Paris', 'Gerard', 'Bondur'),
 ('1143', '1', 'San Francisco', 'Anthony', 'Bow'),
 ('1165', '1', 'San Francisco', 'Leslie', 'Jennings'),
 ('1166', '1', 'San Francisco', 'Leslie', 'Thompson'),
 ('1188', '2', 'Boston', 'Julie', 'Firrelli'),
 ('1216', '2', 'Boston', 'Steve', 'Patterson'),
 ('1286', '3', 'NYC', 'Foon Yue', 'Tseng'),
 ('1323', '3', 'NYC', 'George', 'Vanauf'),
 ('1337', '4', 'Paris', 'Loui', 'Bondur'),
 ('1370', '4', 'Paris', 'Gerard', 'Hernandez'),
 ('1401', '4', 'Paris', 'Pamela', 'Castillo'),
 ('1501', '7', 'London', 'Larry', 'Bott'),
 ('1504', '7', 'London', 'Barry', 'Jones'),
 ('1611', '6', 'Sydney', 'Andy', 'Fixter'),
 ('1612', '6', 'Sydney', 'Peter', 'Marsh'),
 ('1619', '6', 'Sydney', 'Tom', 'King'),
 ('1621', '5', 'Tokyo', 'Mami', 'Nishi'),
 ('1625

In [20]:
#problem # 2
d = pd.DataFrame(list(i) for i in c.fetchall())

In [21]:
d.shape

(0, 0)

In [19]:
d.head()

""


In [6]:
#Your code here



d.columns = [x[0] for x in c.description]
d.head()

ValueError: Length mismatch: Expected axis has 0 elements, new values have 5 elements

## Select the Employee Number, Name (First and Last) and Number of Customers of Employees Who's Customers Have an Average Credit Limit of Over 15K

In [ ]:
#Your code here
c.execute("""select distinct employeeNumber,
                    officeCode,
                    o.city,
                    firstName,
                    lastName
                    from employees e
                    join offices o
                    using(officeCode)
                    join customers c
                    on e.employeeNumber = c.salesRepEmployeeNumber
                    join orders
                    using (customerNumber)
                    join orderdetails
                    using (orderNumber)
                    where productCode in (select productCode
                                                 from products
                                                 join orderdetails
                                                 using (productCode)
                                                 join orders
                                                 using (orderNumber)
                                                 group by 1
                                                 having count(distinct customerNumber) <= 20
                                         );
           """
         )
d2 = pd.DataFrame(c.fetchall())
d2.columns = [x[0] for x in c.description]
print(d2.employeeNumber.unique().shape)

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!